# Дообучение большой языковой модели (LLM).

Ноутбук можно запустить на платформе ML Space, [создав Jupyter Server](https://cloud.ru/docs/aicloud/mlspace/concepts/guides/guides__jupyter/environments__environments__jupyter-server__create-new-jupyter-server.html).

## 1. Установка зависимостей и импорт библиотек

Перед началом работы устанавливаются необходимые пакеты для обработки данных и обучения моделей.

In [3]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes torch transformers peft \
    accelerate datasets loralib==0.1.1 einops==0.6.1 scipy sentencepiece

In [4]:
import os
import torch
import transformers
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/user/conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Загрузка базовой модели

Загружается подходящая предварительно обученная модель, в этом примере выбрана "Intel/neural-chat-7b-v3-1".

In [5]:
MODEL_NAME = "Intel/neural-chat-7b-v3-1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.56s/it]


## 3. Добавление LoRA

Реализуется метод LoRa, который позволяет изменять только небольшую часть параметров модели.

In [7]:
def print_trainable_parameters(model):
  """
  Calculates and displays the total number of model parameters and the number of trainable parameters.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [8]:
model.gradient_checkpointing_enable()

In [9]:
model = prepare_model_for_kbit_training(model)

In [10]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    #target_modules=["query_key_value"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [11]:
print_trainable_parameters(model)

trainable params: 6815744 || all params: 3758886912 || trainables%: 0.18132346515244138


## 4. Дообучение модели

Проверяется, как модель генерирует ответы без дообучения. Затем модель дообучается.

In [12]:
prompt = """
<human>: midjourney prompt for a girl sit on the mountain
<assistant>:
""".strip()

In [13]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [14]:
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)

In [15]:
%%time
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

/home/user/conda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/user/conda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


CPU times: user 3.86 s, sys: 235 ms, total: 4.09 s
Wall time: 4.09 s


In [16]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: midjourney prompt for a girl sit on the mountain
<assistant>: A young girl, dressed in a warm, cozy outfit, sits on a large boulder overlooking a vast, snow-capped mountain range. The sun is setting behind her, casting a golden glow on her face and the surrounding landscape. She gazes into the distance, lost in thought, as the cool breeze gently ruffles her hair.


## 5. Загрузка набора данных

Текстовый запрос к модели генерируется, обрабатывается и передается в модель. 
Для обучения использован датасет [Mid Journey Prompts от Hugging Face](https://huggingface.co/datasets/bittu9988/mid_journey_prompts).

In [17]:
data = load_dataset("bittu9988/mid_journey_prompts")

In [18]:
data

DatasetDict({
    train: Dataset({
        features: ['User', 'Prompt'],
        num_rows: 289
    })
})

In [19]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["User"]}
<assistant>: {data_point["Prompt"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [20]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map: 100%|██████████| 289/289 [00:00<00:00, 4064.93 examples/s]


## 6. Обучение

Задаются параметры модели, и начинается ее дообучение.

In [21]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=8,
      num_train_epochs=4,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
model.config.use_cache = False

In [24]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/user/conda/lib/python3

Step,Training Loss
1,4.427800
2,4.298800
3,4.131700
4,3.993800
5,3.752900
6,3.423000
7,3.130200
8,2.927600
9,2.925300
10,2.731300


TrainOutput(global_step=144, training_loss=1.1761452972681985, metrics={'train_runtime': 289.7835, 'train_samples_per_second': 3.989, 'train_steps_per_second': 0.497, 'total_flos': 3944006887219200.0, 'train_loss': 1.1761452972681985, 'epoch': 3.986159169550173})

## 7. Сохранение модели и инференс

Модель сохраняется в формате, пригодном для переиспользования, и разворачивается.

In [25]:
model.save_pretrained("trained-model")

In [26]:
config = PeftConfig.from_pretrained('./trained-model')
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


In [27]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

In [28]:
model = PeftModel.from_pretrained(model, './trained-model')

In [29]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [30]:
device = "cuda:0"

prompt = """
<human>: midjourney prompt for 
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)

In [31]:
%%time

with torch.no_grad():
  model.config.use_cache = False
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

/home/user/conda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/user/conda/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


CPU times: user 10.6 s, sys: 68 ms, total: 10.6 s
Wall time: 10.6 s


In [32]:
print(tokenizer.decode(outputs[0]))

<s> <human>: midjourney prompt for 
<assistant>: 8k octane render of a realistic furry creature in a forest environment with Pixar render, pencil test, pixar style, character studio, character render, high detail, hyper-realistic, photorealistic, cinematic, 8k, octane render, environment, nature, forest, greens, plants, high detail, 8k, octane render, cinematic, 8k, high detail, hyper-realistic, photorealistic, pixar style, character studio, character render, environment, nature, forest, greens, plants, high detail, 8k, octane render, cinematic, 8k, high detail, hyper-realistic, photorealistic, pixar style, character studio, character render, environment, nature, forest, greens, plants, high detail, 8k, octane render, cinematic, 8k,
